In [1]:
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import ttk
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV

In [2]:
# Tải dữ liệu
file_path = '../Data/clean_data.csv'
data = pd.read_csv(file_path)

In [3]:
# Chia dữ liệu thành các đặc trưng (X) và biến mục tiêu (Y)
X = data.drop(columns=['No', 'Price'])  # Xóa các cột 'No' và 'Price' khỏi dữ liệu, giữ lại các đặc trưng
Y = data['Price']  # Lấy cột 'Price' làm biến mục tiêu


In [4]:
# Chuẩn hóa các biến số
scaler = StandardScaler()  # Khởi tạo bộ chuẩn hóa StandardScaler
X = scaler.fit_transform(X)  # Chuẩn hóa dữ liệu X

In [5]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra (80% cho huấn luyện, 20% cho kiểm tra)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [6]:
# Tối ưu hóa tham số
param_distributions = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [3, 4, 5, 6, 7],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 3, 4, 5],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'learning_rate': [0.01, 0.05, 0.1, 0.2]
}

# Khởi tạo mô hình GradientBoostingRegressor
model = GradientBoostingRegressor(random_state=42)

# Khởi tạo RandomizedSearchCV để tìm kiếm tham số tốt nhất
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_distributions,
                                   n_iter=100, cv=5, n_jobs=-1, scoring='r2', verbose=2, random_state=42)
random_search.fit(X_train, Y_train)  # Huấn luyện RandomizedSearchCV

Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(cv=5, estimator=GradientBoostingRegressor(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'learning_rate': [0.01, 0.05, 0.1, 0.2],
                                        'max_depth': [3, 4, 5, 6, 7],
                                        'min_samples_leaf': [1, 2, 3, 4, 5],
                                        'min_samples_split': [2, 5, 10, 15],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500],
                                        'subsample': [0.6, 0.7, 0.8, 0.9, 1.0]},
                   random_state=42, scoring='r2', verbose=2)

In [7]:
# Sử dụng mô hình với tham số tốt nhất
best_model = random_search.best_estimator_

In [8]:
# Hàm dự đoán giá nhà dựa trên đầu vào của người dùng
def predict_house_price(area, bedrooms, wcs, floors, legal_status, price_per_sqm, district_encoded, month, frontage_1, frontage_0, frontage_missing):
    # Tạo DataFrame từ các tham số đầu vào
    input_data = pd.DataFrame({
        'Area': [area],
        'Bedrooms': [bedrooms],
        'WCs': [wcs],
        'Floors': [floors],
        'Legal_status': [legal_status],
        'Price_per_sqm': [price_per_sqm],
        'District_encoded': [district_encoded],
        'Month': [month],
        'Frontage_1': [frontage_1],
        'Frontage_0': [frontage_0],
        'Frontage_missing': [frontage_missing]
    })
    # Chuẩn hóa dữ liệu đầu vào
    input_data_scaled = scaler.transform(input_data)
    # Dự đoán giá nhà
    prediction = best_model.predict(input_data_scaled)
    return prediction[0]

In [9]:
# Ứng dụng GUI
class HousePriceApp:
    def __init__(self, root):
        self.root = root
        self.root.title("House Price Prediction")  # Đặt tiêu đề cho cửa sổ
        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)  # Gọi hàm on_closing khi cửa sổ bị đóng

        # Tạo các trường nhập liệu
        self.area_label = ttk.Label(root, text="Area")
        self.area_label.grid(row=0, column=0, padx=10, pady=10)
        self.area_entry = ttk.Entry(root)
        self.area_entry.grid(row=0, column=1, padx=10, pady=10)

        self.bedrooms_label = ttk.Label(root, text="Bedrooms")
        self.bedrooms_label.grid(row=1, column=0, padx=10, pady=10)
        self.bedrooms_entry = ttk.Entry(root)
        self.bedrooms_entry.grid(row=1, column=1, padx=10, pady=10)

        self.wcs_label = ttk.Label(root, text="WCs")
        self.wcs_label.grid(row=2, column=0, padx=10, pady=10)
        self.wcs_entry = ttk.Entry(root)
        self.wcs_entry.grid(row=2, column=1, padx=10, pady=10)

        self.floors_label = ttk.Label(root, text="Floors")
        self.floors_label.grid(row=3, column=0, padx=10, pady=10)
        self.floors_entry = ttk.Entry(root)
        self.floors_entry.grid(row=3, column=1, padx=10, pady=10)

        self.legal_status_label = ttk.Label(root, text="Legal Status")
        self.legal_status_label.grid(row=4, column=0, padx=10, pady=10)
        self.legal_status_entry = ttk.Entry(root)
        self.legal_status_entry.grid(row=4, column=1, padx=10, pady=10)

        self.price_per_sqm_label = ttk.Label(root, text="Price per Sqm")
        self.price_per_sqm_label.grid(row=5, column=0, padx=10, pady=10)
        self.price_per_sqm_entry = ttk.Entry(root)
        self.price_per_sqm_entry.grid(row=5, column=1, padx=10, pady=10)

        self.district_encoded_label = ttk.Label(root, text="District Encoded")
        self.district_encoded_label.grid(row=6, column=0, padx=10, pady=10)
        self.district_encoded_entry = ttk.Entry(root)
        self.district_encoded_entry.grid(row=6, column=1, padx=10, pady=10)

        self.month_label = ttk.Label(root, text="Month")
        self.month_label.grid(row=7, column=0, padx=10, pady=10)
        self.month_entry = ttk.Entry(root)
        self.month_entry.grid(row=7, column=1, padx=10, pady=10)

        self.frontage_1_label = ttk.Label(root, text="Frontage 1")
        self.frontage_1_label.grid(row=8, column=0, padx=10, pady=10)
        self.frontage_1_entry = ttk.Entry(root)
        self.frontage_1_entry.grid(row=8, column=1, padx=10, pady=10)

        self.frontage_0_label = ttk.Label(root, text="Frontage 0")
        self.frontage_0_label.grid(row=9, column=0, padx=10, pady=10)
        self.frontage_0_entry = ttk.Entry(root)
        self.frontage_0_entry.grid(row=9, column=1, padx=10, pady=10)

        self.frontage_missing_label = ttk.Label(root, text="Frontage Missing")
        self.frontage_missing_label.grid(row=10, column=0, padx=10, pady=10)
        self.frontage_missing_entry = ttk.Entry(root)
        self.frontage_missing_entry.grid(row=10, column=1, padx=10, pady=10)

        # Tạo nút dự đoán
        self.predict_button = ttk.Button(root, text="Predict Price", command=self.predict_price)
        self.predict_button.grid(row=11, column=0, columnspan=2, padx=10, pady=10)

        # Tạo nhãn để hiển thị kết quả
        self.result_label = ttk.Label(root, text="Predicted Price: ")
        self.result_label.grid(row=12, column=0, columnspan=2, padx=10, pady=10)

    def predict_price(self):
        try:
            # Lấy đầu vào từ người dùng
            area = float(self.area_entry.get())
            bedrooms = int(self.bedrooms_entry.get())
            wcs = int(self.wcs_entry.get())
            floors = int(self.floors_entry.get())
            legal_status = int(self.legal_status_entry.get())
            price_per_sqm = float(self.price_per_sqm_entry.get())
            district_encoded = int(self.district_encoded_entry.get())
            month = int(self.month_entry.get())
            frontage_1 = int(self.frontage_1_entry.get())
            frontage_0 = int(self.frontage_0_entry.get())
            frontage_missing = int(self.frontage_missing_entry.get())

            # Dự đoán giá
            predicted_price = predict_house_price(area, bedrooms, wcs, floors, legal_status, price_per_sqm, district_encoded, month, frontage_1, frontage_0, frontage_missing)
            self.result_label.config(text=f"Predicted Price: {predicted_price}")
        except ValueError:
            self.result_label.config(text="Please enter valid values.")  # Hiển thị thông báo lỗi nếu có lỗi

    def on_closing(self):
        self.root.destroy()  # Đóng cửa sổ
        self.root.quit()  # Thoát ứng dụng


In [10]:
# Hàm chính để chạy ứng dụng
if __name__ == "__main__":
    root = tk.Tk()
    app = HousePriceApp(root)
    root.mainloop()